In [ ]:
%%jupyter
library(Strategus)
library(dplyr)

studyStartDate <- "19000101"
studyEndDate <- "20251231"

In [ ]:
%%jupyter
cohorts_set <- c()
cohortDefinitionSet <- rD2E::get_cohort_definition_set(cohorts_set)

In [ ]:
%%jupyter
# Cohort Generator
cgModuleSettingsCreator <- CohortGeneratorModule$new()
cohortDefinitionShared <- cgModuleSettingsCreator$createCohortSharedResourceSpecifications(cohortDefinitionSet)
cohortGeneratorModuleSpecifications <- cgModuleSettingsCreator$createModuleSpecifications(generateStats = TRUE)

print("Cohort Generator specifications created successfully.")

In [ ]:
%%jupyter
# Target-Comparator pairs
cmTcList <- data.frame(
  targetCohortId = 1,
  targetCohortName = "cancer with chemotherapy",
  comparatorCohortId = 2,
  comparatorCohortName = "cancer with radiotherapy"
)
# Outcome cohort
outcomeCohortId <- 3

# Set time-at-risk
timeAtRisks <- tibble(
  label = c("Kaplan-Meier Analysis"),
  riskWindowStart = 0,
  startAnchor = "cohort start",
  riskWindowEnd = 3650,
  endAnchor = "cohort end"
)

# Define the outcome
outcomeList <- lapply(seq_len(1), function(i) {
  CohortMethod::createOutcome(
    outcomeId = outcomeCohortId,
    outcomeOfInterest = TRUE
  )
})

# Define the T-C-O structure
targetComparatorOutcomesList <- list(
  CohortMethod::createTargetComparatorOutcomes(
    targetId = cmTcList$targetCohortId,
    comparatorId = cmTcList$comparatorCohortId,
    outcomes = outcomeList
  )
)
print("Target-Comparator-Outcomes structure created successfully.")

In [ ]:
%%jupyter
# Setup cohort method module
cmModuleSettingsCreator <- CohortMethodModule$new()

cmAnalysisList <- list(
  CohortMethod::createCmAnalysis(
    analysisId = 1,
    description = "KM analysis with PS Matching",
    getDbCohortMethodDataArgs = CohortMethod::createGetDbCohortMethodDataArgs(
      studyStartDate = studyStartDate,
      studyEndDate = studyEndDate,
      covariateSettings = FeatureExtraction::createDefaultCovariateSettings()
    ),
    createStudyPopArgs = CohortMethod::createCreateStudyPopulationArgs(
        riskWindowStart = timeAtRisks$riskWindowStart,
        startAnchor = timeAtRisks$startAnchor,
        riskWindowEnd = timeAtRisks$riskWindowEnd,
        endAnchor = timeAtRisks$endAnchor,
        minDaysAtRisk = 0
    ),
    createPsArgs = CohortMethod::createCreatePsArgs(),
    matchOnPsArgs = CohortMethod::createMatchOnPsArgs(),
    computeCovariateBalanceArgs = CohortMethod::createComputeCovariateBalanceArgs(),
    fitOutcomeModelArgs = CohortMethod::createFitOutcomeModelArgs(
      modelType = "cox",
      useCovariates = TRUE,
      stratified = TRUE
    ),
  ),
  CohortMethod::createCmAnalysis(
    analysisId = 2,
    description = "KM analysis with No PS Matching",
    getDbCohortMethodDataArgs = CohortMethod::createGetDbCohortMethodDataArgs(
      studyStartDate = studyStartDate,
      studyEndDate = studyEndDate,
      covariateSettings = FeatureExtraction::createDefaultCovariateSettings()
    ),
    createStudyPopArgs = CohortMethod::createCreateStudyPopulationArgs(
        riskWindowStart = timeAtRisks$riskWindowStart,
        startAnchor = timeAtRisks$startAnchor,
        riskWindowEnd = timeAtRisks$riskWindowEnd,
        endAnchor = timeAtRisks$endAnchor,
        minDaysAtRisk = 0
    ),
    computeCovariateBalanceArgs = CohortMethod::createComputeCovariateBalanceArgs(),
    fitOutcomeModelArgs = CohortMethod::createFitOutcomeModelArgs(
      modelType = "cox",
      useCovariates = TRUE
    ),
  )
)
print("Cohort Method analysis specifications created successfully.")

cohortMethodModuleSpecifications <- cmModuleSettingsCreator$createModuleSpecifications(
  cmAnalysisList = cmAnalysisList,
  targetComparatorOutcomesList = targetComparatorOutcomesList
)

In [ ]:
%%jupyter
# Final Analysis Spec
analysisSpecifications <- createEmptyAnalysisSpecificiations() |>
  addSharedResources(cohortDefinitionShared) |>
  addModuleSpecifications(cohortGeneratorModuleSpecifications) |>
  addModuleSpecifications(cohortMethodModuleSpecifications)


In [ ]:
%%jupyter
options <- create_options(upload_results=TRUE, study_id = 'survival_analysis_cohort_method_1') # set a study_id with a unique id
options$study_id = 'survival_analysis_cohort_method_1'
rD2E::run_strategus_flow(analysisSpecification = analysisSpecifications, options = options)